<a href="https://colab.research.google.com/github/Wanita-8943/efficientnet_keras_transfer_learning/blob/master/All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [227]:
import pandas as pd
import shutil

In [228]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [229]:
df = pd.read_csv (r'/content/drive/My Drive/All_Film7.csv')
df

,Fig,Age(year),Class,Filename,Path_filename,Sex,Floder
0,1,7,Y7,V1.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
1,2,7,Y7,V2.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
2,3,7,Y7,V3.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
3,4,7,Y7,V6.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
4,5,7,Y7,V7.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
...,...,...,...,...,...,...,...
1836,143,18,Y18,S264.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1837,144,18,Y18,S268.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1838,145,18,Y18,S270.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1839,146,18,Y18,S274.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both


In [230]:
classes = ['Y7','Y8','Y9','Y10','Y11','Y12','Y13','Y16','Y17','Y18']
len(classes)

10

In [231]:
df.shape

(1841, 7)

In [232]:
print(set(df['Class']))
print(set(df['Sex']))
print(set(df['Floder']))

{'Y10', 'Y17', 'Y7', 'Y13', 'Y11', 'Y8', 'Y16', 'Y9', 'Y12', 'Y18'}
{'เพศหญิง', 'เพศชาย'}
{'Lt', 'Both', 'Rt'}


#Imports

In [233]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

#กำหนดค่าพารามิเตอร์


In [234]:
batch_size = 15

width = 150
height = 150
epochs = 20
NUM_TRAIN = 1800
NUM_TEST = 200
dropout_rate = 0.2
input_shape = (height, width, 3)

#Clone efficientnet repo


In [235]:
#ดึงข้อมูลใน Github มาใช้
import os
%cd /content
if not os.path.isdir("efficientnet_keras_transfer_learning"):
 !git clone https://github.com/Wanita-8943/efficientnet_keras_transfer_learning
%cd efficientnet_keras_transfer_learning/


/content
/content/efficientnet_keras_transfer_learning


In [236]:
# Options: EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3
# Higher the number, the more complex the model is.
from efficientnet import EfficientNetB0 as Net
from efficientnet import center_crop_and_resize, preprocess_input

In [237]:
# loading pretrained conv base model
# โหลดโมเดล มาโดยตัด output ของโมเดลออก เเต่ยังใช้ input อันเดิม
# เเละโหลด weight ของโมเดล มาด้วยที่ชื่อว่า imagenet
conv_base = Net(weights='imagenet', include_top=False, input_shape=input_shape)

In [251]:
conv_base.summary() #ดู Summary

Model: "efficientnet-b0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_585 (Conv2D)            (None, 75, 75, 32)   864         ['input_10[0][0]']               
                                                                                                  
 batch_normalization_441 (Batch  (None, 75, 75, 32)  128         ['conv2d_585[0][0]']             
 Normalization)                                                                                   
                                                                                    

#Download and extract Panoramic dataset

In [238]:
base_dir = '/content/drive/My Drive/Panoramic_Film'
os.makedirs(base_dir, exist_ok=True)

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)
test_dir = os.path.join(base_dir, 'test')
os.makedirs(test_dir, exist_ok=True)

In [239]:
train_Y7_dir = os.path.join(train_dir, 'Y7')
os.makedirs(train_Y7_dir, exist_ok=True)

train_Y8_dir = os.path.join(train_dir, 'Y8')
os.makedirs(train_Y8_dir, exist_ok=True)

train_Y9_dir = os.path.join(train_dir, 'Y9')
os.makedirs(train_Y9_dir, exist_ok=True)

train_Y10_dir = os.path.join(train_dir, 'Y10')
os.makedirs(train_Y10_dir, exist_ok=True)

train_Y11_dir = os.path.join(train_dir, 'Y11')
os.makedirs(train_Y11_dir, exist_ok=True)

train_Y12_dir = os.path.join(train_dir, 'Y12')
os.makedirs(train_Y12_dir, exist_ok=True)

train_Y13_dir = os.path.join(train_dir, 'Y13')
os.makedirs(train_Y13_dir, exist_ok=True)

train_Y16_dir = os.path.join(train_dir, 'Y16')
os.makedirs(train_Y16_dir, exist_ok=True)

train_Y17_dir = os.path.join(train_dir, 'Y17')
os.makedirs(train_Y17_dir, exist_ok=True)

train_Y18_dir = os.path.join(train_dir, 'Y18')
os.makedirs(train_Y18_dir, exist_ok=True)

validation_Y7_dir = os.path.join(validation_dir, 'Y7')
os.makedirs(validation_Y7_dir, exist_ok=True)

validation_Y8_dir = os.path.join(validation_dir, 'Y8')
os.makedirs(validation_Y8_dir, exist_ok=True)

validation_Y9_dir = os.path.join(validation_dir, 'Y9')
os.makedirs(validation_Y9_dir, exist_ok=True)

validation_Y10_dir = os.path.join(validation_dir, 'Y10')
os.makedirs(validation_Y10_dir, exist_ok=True)

validation_Y11_dir = os.path.join(validation_dir, 'Y11')
os.makedirs(validation_Y11_dir, exist_ok=True)

validation_Y12_dir = os.path.join(validation_dir, 'Y12')
os.makedirs(validation_Y12_dir, exist_ok=True)

validation_Y13_dir = os.path.join(validation_dir, 'Y13')
os.makedirs(validation_Y13_dir, exist_ok=True)

validation_Y16_dir = os.path.join(validation_dir, 'Y16')
os.makedirs(validation_Y16_dir, exist_ok=True)

validation_Y17_dir = os.path.join(validation_dir, 'Y17')
os.makedirs(validation_Y17_dir, exist_ok=True)

validation_Y18_dir = os.path.join(validation_dir, 'Y18')
os.makedirs(validation_Y18_dir, exist_ok=True)

test_Y7_dir = os.path.join(test_dir, 'Y7')
os.makedirs(test_Y7_dir, exist_ok=True)

test_Y8_dir = os.path.join(test_dir, 'Y8')
os.makedirs(test_Y8_dir, exist_ok=True)

test_Y9_dir = os.path.join(test_dir, 'Y9')
os.makedirs(test_Y9_dir, exist_ok=True)

test_Y10_dir = os.path.join(test_dir, 'Y10')
os.makedirs(test_Y10_dir, exist_ok=True)

test_Y11_dir = os.path.join(test_dir, 'Y11')
os.makedirs(test_Y11_dir, exist_ok=True)

test_Y12_dir = os.path.join(test_dir, 'Y12')
os.makedirs(test_Y12_dir, exist_ok=True)

test_Y13_dir = os.path.join(test_dir, 'Y13')
os.makedirs(test_Y13_dir, exist_ok=True)

test_Y16_dir = os.path.join(test_dir, 'Y16')
os.makedirs(test_Y16_dir, exist_ok=True)

test_Y17_dir = os.path.join(test_dir, 'Y17')
os.makedirs(test_Y17_dir, exist_ok=True)

test_Y18_dir = os.path.join(test_dir, 'Y18')
os.makedirs(test_Y18_dir, exist_ok=True)


In [240]:
df

,Fig,Age(year),Class,Filename,Path_filename,Sex,Floder
0,1,7,Y7,V1.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
1,2,7,Y7,V2.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
2,3,7,Y7,V3.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
3,4,7,Y7,V6.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
4,5,7,Y7,V7.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
...,...,...,...,...,...,...,...
1836,143,18,Y18,S264.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1837,144,18,Y18,S268.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1838,145,18,Y18,S270.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1839,146,18,Y18,S274.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both


In [241]:
val = df[df['Fig'].between(161,180)]
train = df[df['Fig'].between(1,160)]
test = df[df['Fig'].between(181,200)]

Y7_train = train[train['Class']=='Y7' ]
Y7_path_train = Y7_train['Path_filename'].tolist() 
Y8_train = train[train['Class']=='Y8' ]
Y8_path_train = Y8_train['Path_filename'].tolist() 
Y9_train = train[train['Class']=='Y9' ]
Y9_path_train = Y9_train['Path_filename'].tolist()
Y10_train = train[train['Class']=='Y10' ]
Y10_path_train = Y10_train['Path_filename'].tolist()
Y11_train = train[train['Class']=='Y11' ]
Y11_path_train = Y11_train['Path_filename'].tolist()
Y12_train = train[train['Class']=='Y12' ]
Y12_path_train = Y12_train['Path_filename'].tolist()
Y13_train = train[train['Class']=='Y13' ]
Y13_path_train = Y13_train['Path_filename'].tolist()
Y16_train = train[train['Class']=='Y16' ]
Y16_path_train = Y16_train['Path_filename'].tolist() 
Y17_train = train[train['Class']=='Y17' ]
Y17_path_train = Y17_train['Path_filename'].tolist()
Y18_train = train[train['Class']=='Y18' ]
Y18_path_train = Y18_train['Path_filename'].tolist()

Y7_val = val[val['Class']=='Y7' ]
Y7_path_val = Y7_val['Path_filename'].tolist() 
Y8_val = val[val['Class']=='Y8' ]
Y8_path_val = Y8_val['Path_filename'].tolist() 
Y9_val = val[val['Class']=='Y9' ]
Y9_path_val = Y9_val['Path_filename'].tolist()
Y10_val = val[val['Class']=='Y10' ]
Y10_path_val = Y10_val['Path_filename'].tolist()
Y11_val = val[val['Class']=='Y11' ]
Y11_path_val = Y11_val['Path_filename'].tolist()
Y12_val = val[val['Class']=='Y12' ]
Y12_path_val = Y12_val['Path_filename'].tolist()
Y13_val = val[val['Class']=='Y13' ]
Y13_path_val = Y13_val['Path_filename'].tolist()
Y16_val = val[val['Class']=='Y16' ]
Y16_path_val = Y16_val['Path_filename'].tolist() 
Y17_val = val[val['Class']=='Y17' ]
Y17_path_val = Y17_val['Path_filename'].tolist()
Y18_val = val[val['Class']=='Y18' ]
Y18_path_val = Y18_val['Path_filename'].tolist()

Y7_test = test[test['Class']=='Y7' ]
Y7_path_test = Y7_test['Path_filename'].tolist() 
Y8_test = test[test['Class']=='Y8' ]
Y8_path_test = Y8_test['Path_filename'].tolist() 
Y9_test = test[test['Class']=='Y9' ]
Y9_path_test = Y9_test['Path_filename'].tolist()
Y10_test = test[test['Class']=='Y10' ]
Y10_path_test = Y10_test['Path_filename'].tolist()
Y11_test = test[test['Class']=='Y11' ]
Y11_path_test = Y11_test['Path_filename'].tolist()
Y12_test = test[test['Class']=='Y12' ]
Y12_path_test = Y12_test['Path_filename'].tolist()
Y13_test = test[test['Class']=='Y13' ]
Y13_path_test = Y13_test['Path_filename'].tolist()
Y16_test = test[test['Class']=='Y16' ]
Y16_path_test = Y16_test['Path_filename'].tolist() 
Y17_test = test[test['Class']=='Y17' ]
Y17_path_test = Y17_test['Path_filename'].tolist()
Y18_test = test[test['Class']=='Y18' ]
Y18_path_test = Y18_test['Path_filename'].tolist()

In [242]:
fnames = Y7_path_train
for fname in fnames:
    dst = os.path.join(train_Y7_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = Y8_path_train
for fname in fnames:
    dst = os.path.join(train_Y8_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
    
fnames = Y9_path_train  
for fname in fnames:
    dst = os.path.join(train_Y9_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = Y10_path_train 
for fname in fnames:
    dst = os.path.join(train_Y10_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
    
fnames = Y11_path_train
for fname in fnames:
    dst = os.path.join(train_Y11_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  

fnames = Y12_path_train  
for fname in fnames:
    dst = os.path.join(train_Y12_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  
    
fnames = Y13_path_train
for fname in fnames:
    dst = os.path.join(train_Y13_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

    fnames = Y16_path_train  
for fname in fnames:
    dst = os.path.join(train_Y16_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  

fnames = Y17_path_train
for fname in fnames:
    dst = os.path.join(train_Y17_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  
    
fnames = Y18_path_train
for fname in fnames:
    dst = os.path.join(train_Y18_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

In [243]:
fnames = Y7_path_val
for fname in fnames:
    dst = os.path.join(validation_Y7_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = Y8_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y8_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
    
fnames = Y9_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y9_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = Y10_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y10_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
    
fnames = Y11_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y11_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  

fnames = Y12_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y12_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  
    
fnames = Y13_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y13_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

    fnames = Y16_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y16_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  

fnames = Y17_path_val
for fname in fnames:
    dst = os.path.join(validation_Y17_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  
    
fnames = Y18_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y18_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

In [244]:
fnames = Y7_path_test
for fname in fnames:
    dst = os.path.join(test_Y7_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = Y8_path_test
for fname in fnames:
    dst = os.path.join(test_Y8_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
    
fnames = Y9_path_test
for fname in fnames:
    dst = os.path.join(test_Y9_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = Y10_path_test
for fname in fnames:
    dst = os.path.join(test_Y10_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
    
fnames = Y11_path_test 
for fname in fnames:
    dst = os.path.join(test_Y11_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  

fnames = Y12_path_test  
for fname in fnames:
    dst = os.path.join(test_Y12_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  
    
fnames = Y13_path_test  
for fname in fnames:
    dst = os.path.join(test_Y13_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

    fnames = Y16_path_test  
for fname in fnames:
    dst = os.path.join(test_Y16_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  

fnames = Y17_path_test
for fname in fnames:
    dst = os.path.join(test_Y17_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  
    
fnames = Y18_path_test
for fname in fnames:
    dst = os.path.join(test_Y18_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

In [245]:
print('total training 7years images:', len(os.listdir(train_Y7_dir))) 
print('total training 8years images:', len(os.listdir(train_Y8_dir)))
print('total training 9years images:', len(os.listdir(train_Y9_dir)))
print('total training 10years images:', len(os.listdir(train_Y10_dir))) 
print('total training 11years images:', len(os.listdir(train_Y11_dir)))
print('total training 12years images:', len(os.listdir(train_Y12_dir)))
print('total training 13years images:', len(os.listdir(train_Y13_dir))) 
print('total training 16years images:', len(os.listdir(train_Y16_dir)))
print('total training 17years images:', len(os.listdir(train_Y17_dir)))
print('total training 18years images:', len(os.listdir(train_Y18_dir)),'\n')

print('total validation 7years images:', len(os.listdir(validation_Y7_dir)))
print('total validation 8years images:', len(os.listdir(validation_Y8_dir)))
print('total validation 9years images:', len(os.listdir(validation_Y9_dir)))
print('total validation 10years images:', len(os.listdir(validation_Y10_dir)))
print('total validation 11years images:', len(os.listdir(validation_Y11_dir)))
print('total validation 12years images:', len(os.listdir(validation_Y12_dir)))
print('total validation 13years images:', len(os.listdir(validation_Y13_dir)))
print('total validation 16years images:', len(os.listdir(validation_Y16_dir)))
print('total validation 17years images:', len(os.listdir(validation_Y17_dir)))
print('total validation 18years images:', len(os.listdir(validation_Y18_dir)),'\n')

print('total test 7years images:', len(os.listdir(test_Y7_dir)))
print('total test 8years images:', len(os.listdir(test_Y8_dir)))
print('total test 9years images:', len(os.listdir(test_Y9_dir)))
print('total test 10years images:', len(os.listdir(test_Y10_dir)))
print('total test 11years images:', len(os.listdir(test_Y11_dir)))
print('total test 12years images:', len(os.listdir(test_Y12_dir)))
print('total test 13years images:', len(os.listdir(test_Y13_dir)))
print('total test 16years images:', len(os.listdir(test_Y16_dir)))
print('total test 17years images:', len(os.listdir(test_Y17_dir)))
print('total test 18years images:', len(os.listdir(test_Y18_dir)),'\n')



total training 7years images: 160
total training 8years images: 160
total training 9years images: 160
total training 10years images: 160
total training 11years images: 160
total training 12years images: 160
total training 13years images: 160
total training 16years images: 160
total training 17years images: 160
total training 18years images: 147 

total validation 7years images: 20
total validation 8years images: 20
total validation 9years images: 20
total validation 10years images: 20
total validation 11years images: 20
total validation 12years images: 20
total validation 13years images: 3
total validation 16years images: 20
total validation 17years images: 6
total validation 18years images: 0 

total test 7years images: 20
total test 8years images: 20
total test 9years images: 20
total test 10years images: 20
total test 11years images: 8
total test 12years images: 16
total test 13years images: 0
total test 16years images: 0
total test 17years images: 0
total test 18years images: 0 



In [246]:
# Train ด้วย ImageDataGenerator ของ Keras ซึ่งจะเพิ่มข้อมูลเสริมระหว่างการฝึกเพื่อลดโอกาสเกิด overfitting
#overfitting เกิดจากข้อมูลที่ซับซ้อนกันเกินไป
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255, #โมเดลส่วนใหญ่ต้องใช้ RGB ในช่วง 0–1
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory #ไดเรกเป้าหมาย
        train_dir,
        # รูปภาพทั้งหมดจะถูกปรับขนาดตามความสูงและความกว้างของเป้าหมาย
        target_size=(height, width),
        batch_size=batch_size,
        # Since we use categorical_crossentropy loss, we need categorical labels
        #เนื่องจากเราใช้ categorical_crossentropy loss เราจึงต้องมีป้ายกำกับตามหมวดหมู่
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory( #การดึงภาพจาก Directory มาเข้าโมเดล 
        validation_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

Found 1587 images belonging to 10 classes.
Found 149 images belonging to 10 classes.


In [252]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
# model.add(layers.Flatten(name="flatten"))
if dropout_rate > 0:
    model.add(layers.Dropout(dropout_rate, name="dropout_out"))
# model.add(layers.Dense(256, activation='relu', name="fc1"))
model.add(layers.Dense(10, activation='softmax', name="fc_out"))

In [253]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b0 (Functional  (None, 5, 5, 1280)       4049564   
 )                                                               
                                                                 
 gap (GlobalMaxPooling2D)    (None, 1280)              0         
                                                                 
 dropout_out (Dropout)       (None, 1280)              0         
                                                                 
 fc_out (Dense)              (None, 10)                12810     
                                                                 
Total params: 4,062,374
Trainable params: 12,810
Non-trainable params: 4,049,564
_________________________________________________________________


In [254]:
print('This is the number of trainable layers '
      'before freezing the conv base:', len(model.trainable_weights))

conv_base.trainable = False

print('This is the number of trainable layers '
      'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable layers before freezing the conv base: 2
This is the number of trainable layers after freezing the conv base: 2


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])
history = model.fit_generator(
      train_generator,
      steps_per_epoch= NUM_TRAIN //batch_size,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps= NUM_TEST //batch_size,
      verbose=1,
      use_multiprocessing=True,
      workers=4)